In [6]:
# SQuad dataset
from datasets import load_dataset

squad = load_dataset("rajpurkar/squad_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [26]:
# print(squad["validation"])
validation_set = squad["validation"]
import pandas as pd

# Convert to pandas DataFrame
squad_test = validation_set.to_pandas()

# Display first few rows
print(squad_test.head())


                         id    title  \
0  56ddde6b9a695914005b9628  Normans   
1  56ddde6b9a695914005b9629  Normans   
2  56ddde6b9a695914005b962a  Normans   
3  56ddde6b9a695914005b962b  Normans   
4  56ddde6b9a695914005b962c  Normans   

                                             context  \
0  The Normans (Norman: Nourmands; French: Norman...   
1  The Normans (Norman: Nourmands; French: Norman...   
2  The Normans (Norman: Nourmands; French: Norman...   
3  The Normans (Norman: Nourmands; French: Norman...   
4  The Normans (Norman: Nourmands; French: Norman...   

                                            question  \
0               In what country is Normandy located?   
1                 When were the Normans in Normandy?   
2      From which countries did the Norse originate?   
3                          Who was the Norse leader?   
4  What century did the Normans first gain their ...   

                                             answers  
0  {'text': ['France', 'France

In [77]:
squad_test.columns

Index(['id', 'title', 'context', 'question', 'answers'], dtype='object')

In [11]:
# !pip install datasets

In [12]:
import kagglehub

# Download latest version
sst_path = kagglehub.dataset_download("atulanandjha/stanford-sentiment-treebank-v2-sst2")

print("Path to dataset files:", sst)
# print(sst.head())

Path to dataset files: /root/.cache/kagglehub/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2/versions/30


In [24]:
import pandas as pd
import io

# Path to the CSV file (replace with actual filename in the dataset folder)
# uploaded = files.upload()
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the CSV into a DataFrame



Saving sst2_test.csv to sst2_test (1).csv


In [23]:
sst2_test = pd.read_csv("sst2_test.csv")

# Display the first few rows
print(sst2_test.head())

                                            sentence  label
0                     Effective but too-tepid biopic      2
1  If you sometimes like to go to the movies to h...      3
2  Emerges as something rare , an issue movie tha...      4
3  The film provides some great insight into the ...      2
4  Offers that rare combination of entertainment ...      4


In [29]:
import pandas as pd
import io

# Path to the CSV file (replace with actual filename in the dataset folder)
# uploaded = files.upload()
from google.colab import files

# Upload the file
uploaded = files.upload()

# Load the CSV into a DataFrame



Saving stanford-sentiment-treebank-v2-sst2-metadata.json to stanford-sentiment-treebank-v2-sst2-metadata.json
Saving tokenizer.json to tokenizer.json
Saving tokenizer_config.json to tokenizer_config.json
Saving special_tokens_map.json to special_tokens_map.json
Saving config.json to config.json


In [31]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# from datasets import load_metric
import evaluate
import pandas as pd
from tqdm import tqdm

# Set device to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [37]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Q4

In [75]:
#Load the fine-tuned model
from safetensors.torch import load_file
from transformers import AutoModelForCausalLM, AutoConfig

# Paths
weights_path = "/content/drive/My Drive/model.safetensors"
config_path = "/content/drive/My Drive/config.json"

# Load configuration
config = AutoConfig.from_pretrained(config_path)

# Load weights
state_dict = load_file(weights_path)

# Load model with weights
model = AutoModelForCausalLM.from_config(config)
# model.load_state_dict(state_dict)
# model = model.to("cuda" if torch.cuda.is_available() else "cpu")


In [40]:
model.load_state_dict(state_dict, strict=False)


_IncompatibleKeys(missing_keys=['lm_head.weight'], unexpected_keys=['score.weight'])

In [43]:
import evaluate

# Load metrics
classification_metrics = {
    "accuracy": evaluate.load("accuracy"),
    "precision": evaluate.load("precision"),
    "recall": evaluate.load("recall"),
    "f1": evaluate.load("f1"),
}

qa_metrics = {
    "squad_v2": evaluate.load("squad_v2"),
    "bleu": evaluate.load("bleu"),
    "rouge": evaluate.load("rouge"),
    "meteor": evaluate.load("meteor"),
}


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
#Using fine-tuned  model

In [72]:
# classification
# sst2_test = pd.read_csv("sst2_test.csv")

# Create a classification pipeline
classification_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Predictions and evaluation
true_labels = sst2_test["label"].tolist()
predicted_labels = []
for text in tqdm(sst2_test["sentence"]):
    result = classification_pipeline(text)
    predicted_labels.append(int(result[0]["label"].split("_")[-1]))  # Assuming labels are like "LABEL_0", "LABEL_1"

# # Calculate metrics
# classification_results = {}
# for metric_name, metric in classification_metrics.items():
#     classification_results[metric_name] = metric.compute(predictions=predicted_labels, references=true_labels)

# print("Classification Metrics:", classification_results)


In [68]:
#Finding metrics from above, Classification part on fine-tuned model
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import pandas as pd

# Assuming true_labels and predicted_labels are your lists of true and predicted class labels
classification_results = {}

# Get the classification report as a dictionary (instead of a string)
report = classification_report(true_labels, predicted_labels, output_dict=True)

# Convert the classification report to a pandas DataFrame for a table-like display
report_df = pd.DataFrame(report).transpose()

# Print the classification report in table format
print("Classification Report as Table:")
print(report_df)

# Alternatively, for individual metrics, you can compute and store them in a dictionary
f1 = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)  # Use zero_division=0 or 1
classification_results['f1_score'] = f1

precision = precision_score(true_labels, predicted_labels, average='macro', zero_division=0)  # or 1
classification_results['precision_score'] = precision

recall = recall_score(true_labels, predicted_labels, average='macro', zero_division=0)  # or 1
classification_results['recall_score'] = recall

# Optionally, print individual metrics in table format
classification_results_df = pd.DataFrame(classification_results, index=[0])
print("\nIndividual Metrics as Table:")
print(classification_results_df)


Classification Report as Table:
              precision    recall  f1-score      support
0              0.127059  1.000000  0.225470   270.000000
1              0.000000  0.000000  0.000000   603.000000
2              0.000000  0.000000  0.000000   376.000000
3              0.000000  0.000000  0.000000   491.000000
4              0.000000  0.000000  0.000000   385.000000
accuracy       0.127059  0.127059  0.127059     0.127059
macro avg      0.025412  0.200000  0.045094  2125.000000
weighted avg   0.016144  0.127059  0.028648  2125.000000

Individual Metrics as Table:
   f1_score  precision_score  recall_score
0  0.045094         0.025412           0.2


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [90]:
def custom_qa_pipeline(question, context, max_length=256):
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    # Remove `token_type_ids` if they exist
    inputs.pop("token_type_ids", None)

    # Generate the answer
    outputs = model.generate(
        **inputs, max_length=max_length, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return {"answer": answer}


In [91]:
from tqdm import tqdm

# Prepare input data
qa_input = [
    {"question": row["question"], "context": row["context"]}
    for _, row in squad_test.iterrows()
    if row["question"] and row["context"]  # Ensure non-empty inputs
]

# Get predictions for the entire dataset
predictions = []
for qa in tqdm(qa_input, total=len(qa_input)):
    try:
        result = custom_qa_pipeline(qa["question"], qa["context"])
        predictions.append(result["answer"])
    except Exception as e:
        predictions.append("Error")  # Handle exceptions gracefully
        print(f"Error processing question: {qa['question']}, Exception: {e}")


  0%|          | 9/11873 [06:54<151:16:05, 45.90s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Error processing question: Who was the duke in the battle of Hastings?, Exception: Input length of input_ids is 305, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Error processing question: Who ruled the duchy of Normandy, Exception: Input length of input_ids is 305, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Error processing question: What religion were the Normans, Exception: Input length of input_ids is 302, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Error processing question: What type of major impact did the Norman dynasty have on modern Europe?, Exception: Input length of input_ids is 309, but `max_length` is set to 256. This can lead to unexpected b

  0%|          | 37/11873 [25:30<135:57:37, 41.35s/it]


KeyboardInterrupt: 

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

# Example metric evaluation (adapt to your needs)
true_answers = [row["answers"]["text"][0] for _, row in squad_test.iterrows()]
results = {"rouge": [], "bleu": []}

for pred, true in zip(predictions, true_answers):
    rouge = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True).score(true, pred)["rougeL"].fmeasure
    bleu = sentence_bleu([true.split()], pred.split())
    results["rouge"].append(rouge)
    results["bleu"].append(bleu)

# Print aggregated results
print(f"ROUGE-L: {sum(results['rouge']) / len(results['rouge']):.2%}")
print(f"BLEU: {sum(results['bleu']) / len(results['bleu']):.2%}")


In [ ]:
#Load the pre-trained model

In [ ]:
# Using pre-trained zero-shot model
from safetensors.torch import load_file
from transformers import AutoModelForCausalLM, AutoConfig

# Paths
weights_path = "/content/drive/My Drive/pretrained_model.safetensors"
config_path = "/content/drive/My Drive/pretrained_cofig.json"

# Load configuration
config = AutoConfig.from_pretrained(config_path)

# Load weights
state_dict = load_file(weights_path)

# Load model with weights
pretrained_model = AutoModelForCausalLM.from_config(config)
# model.load_state_dict(state_dict)
# model = model.to("cuda" if torch.cuda.is_available() else "cpu")
pretrained_model.load_state_dict(state_dict, strict=False)





In [ ]:
#Load pre-trained zero-shot model

In [ ]:
# classification

# Create a classification pipeline
classification_pipeline = pipeline("text-classification", model=pretrained_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Predictions and evaluation
true_labels = sst2_test["label"].tolist()
predicted_labels = []
for text in tqdm(sst2_test["sentence"]):
    result = classification_pipeline(text)
    predicted_labels.append(int(result[0]["label"].split("_")[-1]))  # Assuming labels are like "LABEL_0", "LABEL_1"

# Calculate metrics
classification_results = {}
for metric_name, metric in classification_metrics.items():
    classification_results[metric_name] = metric.compute(predictions=predicted_labels, references=true_labels)

print("Classification Metrics:", classification_results)


In [ ]:
# QA using fine-tuned model

# Create a question-answering pipeline
qa_pipeline = pipeline("question-answering", model=pretrained_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Predictions and evaluation
true_answers = []
predicted_answers = []
for i, row in tqdm(squad_test.iterrows(), total=len(squad_test)):
    result = qa_pipeline(question=row["question"], context=row["context"])
    predicted_answers.append(result["answer"])
    true_answers.append({"text": row["answers"], "answer_start": row["answer_start"]})

# Calculate metrics
qa_results = {}
qa_results["exact_match"] = qa_metrics["squad_v2"].compute(predictions=predicted_answers, references=true_answers)
qa_results["f1"] = qa_metrics["f1"].compute(predictions=predicted_answers, references=true_answers)
qa_results["meteor"] = qa_metrics["meteor"].compute(predictions=predicted_answers, references=true_answers)
qa_results["bleu"] = qa_metrics["bleu"].compute(predictions=predicted_answers, references=true_answers)
qa_results["rouge"] = qa_metrics["rouge"].compute(predictions=predicted_answers, references=true_answers)

print("Question-Answering Metrics:", qa_results)


# Q 5

In [74]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

# Model parameters

# Load fine-tuned model (if available)
fine_tuned_params = count_parameters(model)

print(f"Fine-Tuned Model Parameters: {fine_tuned_params}")


Fine-Tuned Model Parameters: 1235814400


In [ ]:
# Total parameters in pre-trained meta-llama/Llama-3.2-1B: 1235.82M (1235824640 parameters)
#So the number of parameters does not match in fine-tuned and pre-trained model

# Total parameters in pre-trained meta-llama/Llama-3.2-1B: 1235.82M (1235824640 parameters)
# Fine-Tuned Model Parameters: 1235814400

#So the number of parameters does not match in fine-tuned and pre-trained model

# Q7

1. Lower or Higher Scores in the Metrics [10 pts]


Higher Scores: Higher metric scores (like F1, BLEU, ROUGE, and Exact Match) indicate better alignment of the model's predictions with the ground truth. This reflects the model's ability to accurately understand and generate contextually relevant answers.
Lower Scores: Lower scores signify either a lack of understanding or issues in the model’s ability to generalize. This could be due to:
Insufficient fine-tuning on the task-specific data.
Lack of diverse data during fine-tuning or pretraining.
Poor handling of ambiguous or adversarial questions.
Rationale:

F1 Score: Indicates precision and recall balance. A high score suggests that the predicted answers capture both relevance and completeness.
BLEU/ROUGE: These measure the overlap between predicted and reference answers. Low scores here might highlight issues with phrasing or inability to capture subtle nuances.
Exact Match: Useful for factual datasets like SQuAD, where the answer must exactly match. Lower EM might indicate paraphrasing or misunderstanding.
2. Understanding the Number of Parameters Between Pretraining and Fine-Tuning [05 pts]


Pretraining Parameters: Large models (e.g., LLaMA) have billions of parameters during pretraining, enabling them to learn general representations from diverse data. This is crucial for capturing broad language understanding.
Fine-Tuning Parameters: Fine-tuning often involves updating only a subset of parameters (e.g., LoRA or adapters) to adapt the general knowledge for specific tasks. In some cases, all parameters may be fine-tuned, but this is computationally expensive.
Rationale:

The number of trainable parameters during fine-tuning directly affects adaptability. Fine-tuning a small subset is computationally efficient but might limit task-specific improvements.
Pretrained models with larger parameter counts usually generalize better but may need extensive fine-tuning to specialize.
3. Performance Difference for Zero-Shot and Fine-Tuned Models [05 pts]


Zero-Shot Performance: A pre-trained model is evaluated directly on a downstream task without any task-specific training. It relies on general language understanding and may perform well on tasks with close alignment to pretraining data.
Fine-Tuned Performance: Fine-tuned models are trained on specific datasets, improving their performance on the task. They learn task-specific nuances, which is critical for structured datasets like SQuAD.